<a href="https://colab.research.google.com/github/parimal-art/intel_unnati_project/blob/main/Virtual_sunglasses_tryon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask
!pip install flask-ngrok

!pip install openvino==2024.6.0
!pip install openvino-dev==2024.6.0
!pip install opencv-python
!pip install numpy

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!mv ngrok /usr/local/bin/
!rm ngrok-stable-linux-amd64.zip

!ngrok authtoken 2ufLwOqmP43tMelIx4XrLKHXaRi_4FyiNTon7k7sThKKaj8WB

import cv2
import numpy as np
from openvino.runtime import Core
from flask import Flask, request, jsonify, render_template, send_file
from flask_ngrok import run_with_ngrok
import os
from io import BytesIO
import base64

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 76.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: networkx
    Found existing installation: networkx 3.4.2
    Uninstalling networkx-3.4.2:
      Successfully uninstalled networkx-3.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nx-cugraph-cu12 25.2.0 requires networkx>=3.2, but you have networkx 3.1 which is incompatible.
torch 2.6.0+cu124 requires nv

In [2]:
!omz_downloader --name face-detection-adas-0001 --precision FP16 -o ./models

core = Core()
model = core.read_model(model="./models/intel/face-detection-adas-0001/FP16/face-detection-adas-0001.xml")
compiled_model = core.compile_model(model=model, device_name="CPU")

input_layer = compiled_model.input(0)
output_layer = compiled_model.output(0)

################|| Downloading face-detection-adas-0001 ||################

========== Downloading models/intel/face-detection-adas-0001/FP16/face-detection-adas-0001.xml
... 100%, 304 KB, 858 KB/s, 0 seconds passed

========== Downloading models/intel/face-detection-adas-0001/FP16/face-detection-adas-0001.bin
... 100%, 2056 KB, 2927 KB/s, 0 seconds passed



In [3]:
!rm -f /usr/local/bin/ngrok
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xvzf ngrok-v3-stable-linux-amd64.tgz
!mv ngrok /usr/local/bin/
!rm ngrok-v3-stable-linux-amd64.tgz
!ngrok --version

--2025-03-31 13:13:18--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 13.248.244.96, 35.71.179.82, 75.2.60.68, ...
Connecting to bin.equinox.io (bin.equinox.io)|13.248.244.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9372184 (8.9M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz’

ngrok-v3-stable-lin 100%[===================>]   8.94M  13.4MB/s    in 0.7s    

2025-03-31 13:13:19 (13.4 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz’ saved [9372184/9372184]

ngrok
ngrok version 3.22.0


In [4]:
!ngrok authtoken 2ufLwOqmP43tMelIx4XrLKHXaRi_4FyiNTon7k7sThKKaj8WB
!cat /root/.ngrok2/ngrok.yml

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
Default version saved to configuration file: /root/.ngrok2/ngrok.yml
authtoken: 2ufLwOqmP43tMelIx4XrLKHXaRi_4FyiNTon7k7sThKKaj8WB
version: "2"


In [5]:
import os

# Python script to create and save the HTML file
html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Sunglasses Try-On Studio</title>
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }

        body {
            font-family: 'Montserrat', sans-serif;
            background: linear-gradient(45deg, #1a1a2e 0%, #16213e 100%);
            color: #e0e0e0;
            min-height: 100vh;
            padding: 40px;
            overflow-x: hidden;
        }

        .container {
            max-width: 1000px;
            margin: 0 auto;
            background: rgba(255, 255, 255, 0.05);
            border-radius: 20px;
            padding: 40px;
            backdrop-filter: blur(10px);
            box-shadow: 0 8px 32px rgba(0, 0, 0, 0.2);
            position: relative;
            overflow: hidden;
        }

        .container::before {
            content: '';
            position: absolute;
            top: -50%;
            left: -50%;
            width: 200%;
            height: 200%;
            background: radial-gradient(circle, rgba(255, 255, 255, 0.1) 0%, transparent 70%);
            animation: rotateGlow 15s linear infinite;
        }

        h1 {
            font-size: 3em;
            color: #00d4ff;
            text-transform: uppercase;
            letter-spacing: 3px;
            margin-bottom: 40px;
            text-shadow: 0 0 10px rgba(0, 212, 255, 0.5);
            animation: glow 2s ease-in-out infinite alternate;
            text-align: center;
        }

        h2 {
            font-size: 1.8em;
            color: #fff;
            margin: 30px 0 20px;
            position: relative;
            padding-bottom: 10px;
        }

        h2::after {
            content: '';
            position: absolute;
            bottom: 0;
            left: 0;
            width: 50px;
            height: 3px;
            background: #00d4ff;
            transition: width 0.4s ease;
        }

        h2:hover::after {
            width: 100%;
        }

        .section {
            background: rgba(255, 255, 255, 0.08);
            border-radius: 15px;
            padding: 25px;
            margin: 20px 0;
            transition: all 0.4s ease;
            position: relative;
            overflow: hidden;
        }

        .section:hover {
            transform: translateY(-10px);
            box-shadow: 0 10px 20px rgba(0, 0, 0, 0.3);
            background: rgba(255, 255, 255, 0.12);
        }

        .file-upload-wrapper {
            position: relative;
            display: inline-block;
            width: 100%;
            max-width: 400px;
            margin: 15px 0;
        }

        input[type="file"] {
            width: 100%;
            padding: 15px 20px;
            background: rgba(255, 255, 255, 0.1);
            border-radius: 10px;
            color: #fff;
            border: 2px dashed rgba(0, 212, 255, 0.5);
            cursor: pointer;
            font-size: 1em;
            transition: all 0.3s ease;
            opacity: 0;
            position: absolute;
            top: 0;
            left: 0;
            height: 100%;
        }

        .file-upload-label {
            display: block;
            padding: 15px 20px;
            background: rgba(255, 255, 255, 0.15);
            border-radius: 10px;
            color: #00d4ff;
            border: 2px dashed rgba(0, 212, 255, 0.5);
            text-align: center;
            cursor: pointer;
            transition: all 0.3s ease;
            font-weight: 500;
        }

        .file-upload-wrapper:hover .file-upload-label {
            background: rgba(255, 255, 255, 0.25);
            border-color: rgba(0, 212, 255, 0.8);
            color: #fff;
        }

        .file-name {
            display: block;
            margin-top: 10px;
            color: #e0e0e0;
            font-size: 0.9em;
            word-break: break-all;
        }

        button {
            padding: 14px 40px;
            background: linear-gradient(45deg, #00d4ff, #007bff);
            color: white;
            border: none;
            border-radius: 50px;
            font-size: 1.1em;
            cursor: pointer;
            transition: all 0.4s ease;
            box-shadow: 0 5px 15px rgba(0, 212, 255, 0.4);
        }

        button:hover {
            transform: translateY(-3px);
            box-shadow: 0 8px 20px rgba(0, 212, 255, 0.6);
            background: linear-gradient(45deg, #007bff, #00d4ff);
        }

        button:active {
            transform: scale(0.95);
        }

        img, video, canvas {
            max-width: 100%;
            border-radius: 15px;
            margin: 20px 0;
            border: 2px solid rgba(255, 255, 255, 0.1);
            transition: all 0.5s ease;
            opacity: 0;
        }

        .visible {
            opacity: 1;
        }

        .image-comparison {
            display: flex;
            justify-content: space-between;
            gap: 20px;
            flex-wrap: wrap;
        }

        .image-comparison img {
            max-width: 48%;
            flex: 1;
        }

        .loading {
            display: none;
            font-size: 1.2em;
            color: #00d4ff;
            margin: 15px 0;
            animation: pulse 1.5s infinite;
        }

        @keyframes glow {
            from { text-shadow: 0 0 10px rgba(0, 212, 255, 0.5); }
            to { text-shadow: 0 0 20px rgba(0, 212, 255, 0.8); }
        }

        @keyframes rotateGlow {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }

        @keyframes pulse {
            0% { opacity: 0.6; }
            50% { opacity: 1; }
            100% { opacity: 0.6; }
        }

        .tooltip {
            position: absolute;
            top: 10px;
            right: 10px;
            background: rgba(0, 0, 0, 0.8);
            color: #fff;
            padding: 5px 10px;
            border-radius: 5px;
            font-size: 0.9em;
            opacity: 0;
            transition: opacity 0.3s ease;
        }

        .section:hover .tooltip {
            opacity: 1;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Sunglasses Try-On Studio</h1>

        <!-- Sunglasses Upload Section -->
        <div class="section">
            <h2>Select Your Shades</h2>
            <span class="tooltip">Upload a sunglasses image</span>
            <div class="file-upload-wrapper">
                <input type="file" id="sunglasses-upload" accept="image/*" onchange="showFileName('sunglasses-upload', 'sunglasses-file-name')">
                <label for="sunglasses-upload" class="file-upload-label">Choose Sunglasses Image</label>
                <span id="sunglasses-file-name" class="file-name">No file chosen</span>
            </div>
            <img id="sunglasses-preview" src="" alt="Sunglasses Preview">
            <div class="loading" id="sunglasses-loading">Processing Shades...</div>
            <button onclick="uploadSunglasses()">Upload Sunglasses</button>
        </div>

        <!-- Photo Upload Section -->
        <div class="section">
            <h2>Try with Photo</h2>
            <span class="tooltip">Upload your face photo</span>
            <div class="file-upload-wrapper">
                <input type="file" id="photo-upload" accept="image/*" onchange="showFileName('photo-upload', 'photo-file-name')">
                <label for="photo-upload" class="file-upload-label">Choose Your Photo</label>
                <span id="photo-file-name" class="file-name">No file chosen</span>
            </div>
            <div class="image-comparison">
                <img id="preview" src="" alt="Photo Preview">
                <img id="result" src="" alt="Result">
            </div>
            <div class="loading" id="photo-loading">Applying Sunglasses...</div>
            <button onclick="uploadPhoto()">Try On Now</button>
        </div>

        <!-- Webcam Section -->
        <div class="section">
            <h2>Live Mirror Mode</h2>
            <span class="tooltip">Use your webcam for real-time try-on</span>
            <button onclick="startWebcam()">Start Mirror</button>
            <button onclick="captureFrame()" id="capture-button" style="display: none;">Capture</button>
            <button onclick="stopWebcam()" id="stop-webcam" style="display: none;">Stop Mirror</button>
            <video id="webcam" autoplay playsinline></video>
            <canvas id="canvas" style="display: none;"></canvas>
            <div class="image-comparison">
                <img id="webcam-capture" src="" alt="Captured Frame">
                <img id="webcam-preview" src="" alt="Webcam Preview">
            </div>
            <div class="loading" id="webcam-loading">Processing Capture...</div>
        </div>
    </div>

    <script>
        // Function to show file name when a file is selected
        function showFileName(inputId, displayId) {
            const fileInput = document.getElementById(inputId);
            const fileNameDisplay = document.getElementById(displayId);
            if (fileInput.files.length > 0) {
                fileNameDisplay.textContent = fileInput.files[0].name;
            } else {
                fileNameDisplay.textContent = "No file chosen";
            }
        }

        // Sunglasses Upload Logic
        function uploadSunglasses() {
            const fileInput = document.getElementById('sunglasses-upload');
            const preview = document.getElementById('sunglasses-preview');
            const loading = document.getElementById('sunglasses-loading');

            if (!fileInput.files[0]) {
                alert("Please select a sunglasses image!");
                return;
            }

            loading.style.display = 'block';
            preview.src = URL.createObjectURL(fileInput.files[0]);
            preview.classList.add('visible');

            const formData = new FormData();
            formData.append('sunglasses', fileInput.files[0]);

            fetch('/upload_sunglasses', {
                method: 'POST',
                body: formData
            })
            .then(response => response.json())
            .then(data => {
                console.log('Sunglasses upload response:', data);  // Log response
                loading.style.display = 'none';
                if (data.error) {
                    alert(data.error);
                } else {
                    preview.src = 'data:image/png;base64,' + data.sunglasses;
                    preview.classList.add('visible');
                    console.log('Sunglasses uploaded successfully.');
                }
            })
            .catch(error => {
                loading.style.display = 'none';
                console.error('Error uploading sunglasses:', error);
            });
        }

        // Photo Upload Logic
        function uploadPhoto() {
            const fileInput = document.getElementById('photo-upload');
            const preview = document.getElementById('preview');
            const result = document.getElementById('result');
            const loading = document.getElementById('photo-loading');

            if (!fileInput.files[0]) {
                alert("Please select a photo!");
                return;
            }

            loading.style.display = 'block';
            preview.src = URL.createObjectURL(fileInput.files[0]);
            preview.classList.add('visible');

            const formData = new FormData();
            formData.append('file', fileInput.files[0]);

            fetch('/upload', {
                method: 'POST',
                body: formData
            })
            .then(response => response.json())
            .then(data => {
                console.log('Photo upload response:', data);  // Log response
                loading.style.display = 'none';
                if (data.error) {
                    alert(data.error);
                } else if (data.image) {
                    result.src = 'data:image/png;base64,' + data.image;
                    result.classList.add('visible');
                    console.log('Photo processed successfully and displayed.');
                } else {
                    console.log('No image data in response.');
                }
            })
            .catch(error => {
                loading.style.display = 'none';
                console.error('Error uploading photo:', error);
            });
        }

        // Webcam Logic
        let stream;
        let video;
        let canvas;

        function startWebcam() {
            video = document.getElementById('webcam');
            canvas = document.getElementById('canvas');
            const preview = document.getElementById('webcam-preview');
            const capturedFrame = document.getElementById('webcam-capture');
            const stopButton = document.getElementById('stop-webcam');
            const captureButton = document.getElementById('capture-button');

            navigator.mediaDevices.getUserMedia({ video: { width: 640, height: 480 } })
                .then(s => {
                    stream = s;
                    video.srcObject = stream;
                    video.classList.add('visible');
                    stopButton.style.display = 'inline-block';
                    captureButton.style.display = 'inline-block';

                    video.onloadedmetadata = () => {
                        canvas.width = video.videoWidth;
                        canvas.height = video.videoHeight;
                        console.log('Video dimensions:', video.videoWidth, 'x', video.videoHeight);
                    };
                })
                .catch(error => {
                    console.error('Webcam Error:', error);
                    alert('Could not access webcam. Please check permissions.');
                });
        }

        function captureFrame() {
            const capturedFrame = document.getElementById('webcam-capture');
            const preview = document.getElementById('webcam-preview');
            const loading = document.getElementById('webcam-loading');
            const context = canvas.getContext('2d');

            loading.style.display = 'block';
            context.drawImage(video, 0, 0, canvas.width, canvas.height);
            const imageData = canvas.toDataURL('image/png');
            capturedFrame.src = imageData;  // Display captured frame
            capturedFrame.classList.add('visible');
            console.log('Captured frame:', imageData.substring(0, 50));

            fetch('/webcam', {
                method: 'POST',
                headers: { 'Content-Type': 'application/json' },
                body: JSON.stringify({ image: imageData })
            })
            .then(response => response.json())
            .then(data => {
                console.log('Webcam response:', data);  // Log backend response
                loading.style.display = 'none';
                if (data.error) {
                    alert(data.error);
                } else if (data.image) {
                    preview.src = 'data:image/png;base64,' + data.image;
                    preview.classList.add('visible');
                    console.log('Preview updated with processed frame.');
                } else {
                    console.log('No image data in response.');
                }
            })
            .catch(error => {
                loading.style.display = 'none';
                console.error('Error processing captured frame:', error);
            });
        }

        function stopWebcam() {
            const video = document.getElementById('webcam');
            const capturedFrame = document.getElementById('webcam-capture');
            const preview = document.getElementById('webcam-preview');
            const stopButton = document.getElementById('stop-webcam');
            const captureButton = document.getElementById('capture-button');
            const loading = document.getElementById('webcam-loading');

            stream.getTracks().forEach(track => track.stop());
            video.classList.remove('visible');
            capturedFrame.classList.remove('visible');
            preview.classList.remove('visible');
            stopButton.style.display = 'none';
            captureButton.style.display = 'none';
            loading.style.display = 'none';
        }
    </script>
</body>
</html>
"""

# Create the 'templates' directory if it doesn't exist
os.makedirs('templates', exist_ok=True)

# Save the HTML file
with open('templates/index.html', 'w') as f:
    f.write(html_content)

print("File 'templates/index.html' has been created successfully!")

File 'templates/index.html' has been created successfully!


In [6]:
!wget https://storage.openvinotoolkit.org/repositories/open_model_zoo/2022.1/models_bin/3/face-detection-adas-0001/FP32/face-detection-adas-0001.xml
!wget https://storage.openvinotoolkit.org/repositories/open_model_zoo/2022.1/models_bin/3/face-detection-adas-0001/FP32/face-detection-adas-0001.bin
!ls -l face-detection-adas-0001.xml face-detection-adas-0001.bin

--2025-03-31 13:13:20--  https://storage.openvinotoolkit.org/repositories/open_model_zoo/2022.1/models_bin/3/face-detection-adas-0001/FP32/face-detection-adas-0001.xml
Resolving storage.openvinotoolkit.org (storage.openvinotoolkit.org)... 18.238.238.38, 18.238.238.53, 18.238.238.123, ...
Connecting to storage.openvinotoolkit.org (storage.openvinotoolkit.org)|18.238.238.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 286610 (280K) [application/xml]
Saving to: ‘face-detection-adas-0001.xml’

face-detection-adas 100%[===================>] 279.89K   784KB/s    in 0.4s    

2025-03-31 13:13:21 (784 KB/s) - ‘face-detection-adas-0001.xml’ saved [286610/286610]

--2025-03-31 13:13:21--  https://storage.openvinotoolkit.org/repositories/open_model_zoo/2022.1/models_bin/3/face-detection-adas-0001/FP32/face-detection-adas-0001.bin
Resolving storage.openvinotoolkit.org (storage.openvinotoolkit.org)... 18.238.238.38, 18.238.238.53, 18.238.238.123, ...
Connecting to stora

In [7]:
!pip install rembg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.9 MB/s eta 0:00:00


In [8]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.7 MB/s eta 0:00:00


In [9]:
!pip install onnxruntime

In [10]:
!pip install rembg --force-reinstall

  Using cached rembg-2.0.65-py3-none-any.whl.metadata (20 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.3 MB/s eta 0:00:00
  Using cached PyMatting-1.1.13-py3-none-any.whl.metadata (7.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.4 MB/s eta 0:00:00
Using cached rembg-2.0.65-py3-none-any.whl (41 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.5/88.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 4.2 MB/s eta 0:00:00
Using cached PyMatting-1.1.13-py3-none-any.whl (54 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 

In [3]:
import subprocess
import time
import requests

ngrok_process = subprocess.Popen(['ngrok', 'http', '5000'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

max_attempts = 10
for attempt in range(max_attempts):
    try:
        time.sleep(5)
        ngrok_url = requests.get('http://localhost:4040/api/tunnels').json()['tunnels'][0]['public_url']
        print(f"Ngrok Tunnel URL: {ngrok_url}")
        break
    except Exception as e:
        print(f"Attempt {attempt + 1}/{max_attempts} - Error fetching Ngrok URL: {e}")
        if attempt == max_attempts - 1:
            print("Ngrok process output:", ngrok_process.stderr.read().decode())
            ngrok_process.terminate()

Ngrok Tunnel URL: https://2172-34-82-160-228.ngrok-free.app


In [4]:
import numpy as np
import cv2
from flask import Flask, request, jsonify, render_template
import base64
from flask_ngrok import run_with_ngrok
from openvino.runtime import Core
from rembg import remove
from PIL import Image
import io

app = Flask(__name__)
run_with_ngrok(app)

!mkdir -p templates
!mkdir -p static

global_sunglasses_rgb = None
global_sunglasses_alpha = None

core = Core()
model_path = "/content/face-detection-adas-0001.xml"
model = core.read_model(model_path)
compiled_model = core.compile_model(model, "CPU")
input_layer = compiled_model.input(0)
output_layer = compiled_model.output(0)

def process_image(image, sunglasses_rgb, sunglasses_alpha):
    print("Processing image with shape:", image.shape)
    _, _, h, w = input_layer.shape
    print(f"Resizing image to model input: {w}x{h}")
    blob = cv2.resize(image, (w, h))
    blob = blob.transpose((2, 0, 1))
    blob = blob.reshape((1, 3, h, w))
    outputs = compiled_model([blob])[output_layer]
    detections = outputs
    face_detected = False
    for detection in detections[0][0]:
        confidence = detection[2]
        print(f"Detection confidence: {confidence}")
        if confidence > 0.5:
            xmin = int(detection[3] * image.shape[1])
            ymin = int(detection[4] * image.shape[0])
            xmax = int(detection[5] * image.shape[1])
            ymax = int(detection[6] * image.shape[0])
            xmin = max(0, xmin)
            ymin = max(0, ymin)
            xmax = min(image.shape[1], xmax)
            ymax = min(image.shape[0], ymax)
            print(f"Face detected at: ({xmin}, {ymin}, {xmax}, {ymax})")
            face_detected = True
            break
    if not face_detected:
        print("No face detected in the image.")
        return image
    face_width = xmax - xmin
    face_height = ymax - ymin
    print(f"Face dimensions: {face_width}x{face_height}")
    eyes_width = int(face_width * 0.95)
    eyes_x = xmin + int(face_width * 0.02)
    eyes_y = ymin + int(face_height * 0.29)
    print(f"Estimated eyes position: ({eyes_x}, {eyes_y}), width: {eyes_width}")
    sunglasses_aspect_ratio = sunglasses_rgb.shape[0] / sunglasses_rgb.shape[1]
    sunglasses_height = int(eyes_width * sunglasses_aspect_ratio)
    sunglasses_resized = cv2.resize(sunglasses_rgb, (eyes_width, sunglasses_height), interpolation=cv2.INTER_AREA)
    alpha_resized = cv2.resize(sunglasses_alpha, (eyes_width, sunglasses_height), interpolation=cv2.INTER_AREA)
    print("Sunglasses resized to:", sunglasses_resized.shape)
    alpha_resized = alpha_resized / 255.0 if alpha_resized.max() > 1 else alpha_resized
    print("Alpha channel normalized.")
    start_y = max(eyes_y, 0)
    end_y = min(eyes_y + sunglasses_resized.shape[0], image.shape[0])
    start_x = max(eyes_x, 0)
    end_x = min(eyes_x + sunglasses_resized.shape[1], image.shape[1])
    roi_height = end_y - start_y
    roi_width = end_x - start_x
    sunglasses_resized = sunglasses_resized[:roi_height, :roi_width]
    alpha_resized = alpha_resized[:roi_height, :roi_width]
    print("Adjusted ROI size:", (roi_height, roi_width))
    roi = image[start_y:end_y, start_x:end_x]
    for c in range(3):
        roi[:, :, c] = (alpha_resized * sunglasses_resized[:, :, c] + (1 - alpha_resized) * roi[:, :, c]).astype(np.uint8)
    image[start_y:end_y, start_x:end_x] = roi
    print("Overlay completed.")
    return image

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/upload_sunglasses', methods=['POST'])
def upload_sunglasses():
    global global_sunglasses_rgb, global_sunglasses_alpha
    file = request.files['sunglasses']
    print("Converting uploaded sunglasses image to PIL format...")
    input_image = Image.open(file)
    print("Sunglasses image loaded with format:", input_image.format)
    has_transparency = False
    if input_image.format == "PNG" and input_image.mode == "RGBA":
        print("Image is PNG with RGBA mode, checking for transparency...")
        image_np = np.array(input_image)
        alpha_channel = image_np[:, :, 3]
        if np.any(alpha_channel < 255):
            print("Transparency detected in alpha channel, skipping background removal.")
            has_transparency = True
        else:
            print("No transparency detected in alpha channel, will remove background.")
    if has_transparency:
        output_image = input_image
    else:
        print("Removing background from sunglasses image...")
        output_image = remove(input_image)
        print("Background removed using rembg.")
    print("Converting PIL image to numpy array...")
    output_image_np = np.array(output_image)
    print("Extracting RGB and alpha channels...")
    if output_image_np.shape[-1] == 4:
        global_sunglasses_rgb = output_image_np[:, :, :3]
        global_sunglasses_alpha = output_image_np[:, :, 3]
    else:
        global_sunglasses_rgb = output_image_np
        global_sunglasses_alpha = np.ones(output_image_np.shape[:2], dtype=np.uint8) * 255
    if global_sunglasses_rgb.shape[-1] == 3:
        print("Ensuring RGB format for sunglasses image...")
        global_sunglasses_rgb = cv2.cvtColor(global_sunglasses_rgb, cv2.COLOR_BGR2RGB)
    print("Cropping sunglasses image...")
    non_zero = np.where(global_sunglasses_alpha > 0)
    if len(non_zero[0]) > 0 and len(non_zero[1]) > 0:
        y_min, y_max = np.min(non_zero[0]), np.max(non_zero[0])
        x_min, x_max = np.min(non_zero[1]), np.max(non_zero[1])
        padding = 5
        y_min = max(0, y_min - padding)
        y_max = min(global_sunglasses_alpha.shape[0], y_max + padding)
        x_min = max(0, x_min - padding)
        x_max = min(global_sunglasses_alpha.shape[1], x_max + padding)
        global_sunglasses_rgb = global_sunglasses_rgb[y_min:y_max, x_min:x_max]
        global_sunglasses_alpha = global_sunglasses_alpha[y_min:y_max, x_min:x_max]
        print(f"Cropped sunglasses to bounding box: ({x_min}, {y_min}, {x_max}, {y_max})")
    else:
        print("No non-transparent pixels found, skipping cropping.")
    print("Sunglasses processed with shape:", global_sunglasses_rgb.shape)
    print("Encoding sunglasses for preview...")
    sunglasses_rgb_for_encoding = cv2.cvtColor(global_sunglasses_rgb, cv2.COLOR_RGB2BGR)
    _, buffer = cv2.imencode('.png', sunglasses_rgb_for_encoding)
    sunglasses_base64 = base64.b64encode(buffer).decode('utf-8')
    print("Sunglasses encoded for preview.")
    return jsonify({'sunglasses': sunglasses_base64})

@app.route('/upload', methods=['POST'])
def upload():
    global global_sunglasses_rgb, global_sunglasses_alpha
    if global_sunglasses_rgb is None or global_sunglasses_alpha is None:
        print("Sunglasses not uploaded yet.")
        return jsonify({'error': 'Please upload sunglasses first!'})
    file = request.files['file']
    image = cv2.imdecode(np.frombuffer(file.read(), np.uint8), cv2.IMREAD_COLOR)
    if image is None:
        print("Failed to decode uploaded photo.")
        return jsonify({'error': 'Failed to decode uploaded photo.'})
    print("Photo uploaded with shape:", image.shape)
    processed_image = process_image(image, global_sunglasses_rgb, global_sunglasses_alpha)
    _, buffer = cv2.imencode('.png', processed_image)
    if buffer is None:
        print("Failed to encode processed image.")
        return jsonify({'error': 'Failed to encode processed image.'})
    image_base64 = base64.b64encode(buffer).decode('utf-8')
    print("Processed image encoded for response.")
    return jsonify({'image': image_base64})

@app.route('/webcam', methods=['POST'])
def webcam():
    global global_sunglasses_rgb, global_sunglasses_alpha
    if global_sunglasses_rgb is None or global_sunglasses_alpha is None:
        print("Sunglasses not uploaded yet.")
        return jsonify({'error': 'Please upload sunglasses first!'})
    data = request.json['image']
    print("Received webcam frame.")
    image_data = base64.b64decode(data.split(',')[1])
    image = cv2.imdecode(np.frombuffer(image_data, np.uint8), cv2.IMREAD_COLOR)
    if image is None:
        print("Failed to decode webcam frame.")
        return jsonify({'error': 'Failed to decode webcam frame.'})
    print("Webcam frame decoded with shape:", image.shape)
    processed_image = process_image(image, global_sunglasses_rgb, global_sunglasses_alpha)
    _, buffer = cv2.imencode('.png', processed_image)
    if buffer is None:
        print("Failed to encode processed image.")
        return jsonify({'error': 'Failed to encode processed image.'})
    image_base64 = base64.b64encode(buffer).decode('utf-8')
    print("Processed image encoded for response.")
    return jsonify({'image': image_base64})

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://2172-34-82-160-228.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [31/Mar/2025 13:19:52] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/Mar/2025 13:19:53] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [31/Mar/2025 13:20:01] "POST /upload_sunglasses HTTP/1.1" 200 -


Converting uploaded sunglasses image to PIL format...
Sunglasses image loaded with format: PNG
Image is PNG with RGBA mode, checking for transparency...
Transparency detected in alpha channel, skipping background removal.
Converting PIL image to numpy array...
Extracting RGB and alpha channels...
Ensuring RGB format for sunglasses image...
Cropping sunglasses image...
Cropped sunglasses to bounding box: (10, 6, 987, 364)
Sunglasses processed with shape: (358, 977, 3)
Encoding sunglasses for preview...
Sunglasses encoded for preview.


...

Received webcam frame.
Webcam frame decoded with shape: (480, 640, 3)
Processing image with shape: (480, 640, 3)
Resizing image to model input: 672x384
Detection confidence: 0.9998705387115479
Face detected at: (276, 216, 410, 366)
Face dimensions: 134x150
Estimated eyes position: (278, 259), width: 127
Sunglasses resized to: (46, 127, 3)
Alpha channel normalized.
Adjusted ROI size: (46, 127)
Overlay completed.
Processed image encoded for response.


INFO:werkzeug:127.0.0.1 - - [31/Mar/2025 13:20:43] "POST /webcam HTTP/1.1" 200 -


Received webcam frame.
Webcam frame decoded with shape: (480, 640, 3)
Processing image with shape: (480, 640, 3)
Resizing image to model input: 672x384
Detection confidence: 0.999140739440918
Face detected at: (283, 213, 413, 353)
Face dimensions: 130x140
Estimated eyes position: (285, 253), width: 123
Sunglasses resized to: (45, 123, 3)
Alpha channel normalized.
Adjusted ROI size: (45, 123)
Overlay completed.
Processed image encoded for response.


INFO:werkzeug:127.0.0.1 - - [31/Mar/2025 13:20:58] "POST /webcam HTTP/1.1" 200 -


Received webcam frame.
Webcam frame decoded with shape: (480, 640, 3)
Processing image with shape: (480, 640, 3)
Resizing image to model input: 672x384
Detection confidence: 0.9999383687973022
Face detected at: (270, 212, 404, 365)
Face dimensions: 134x153
Estimated eyes position: (272, 256), width: 127
Sunglasses resized to: (46, 127, 3)
Alpha channel normalized.
Adjusted ROI size: (46, 127)
Overlay completed.
Processed image encoded for response.
